In [55]:
from sklearn.decomposition import randomized_svd
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [56]:
#user = pd.read_csv('../Data/whyout_data/user.csv') # shape(31178,13), user idx에서 2354개가 비어있음
place = pd.read_csv('../Data/whyout_data/place.csv') # shape(4697,10), place idx에서 23개가 비어있음
#product = pd.read_csv('../Data/whyout_data/product.csv') # shape(5834,11), product idx에서 538개가 비어있음
#video = pd.read_csv('../Data/whyout_data/video.csv') # shape(3250, 9), video idx에서 315개가 비어있음
#user_place = pd.read_csv('../Data/whyout_data/user_place.csv') # shape (31176,4697) 유저 x 장소 아이템 # 8756명이 장소를 평가하지 않음
#user_product = pd.read_csv('../Data/whyout_data/user_product.csv') # shape(31176,5834) 유저 x 상품 아이템
#user_video = pd.read_csv('../Data/whyout_data/user_video.csv') # shape (31177, 3250) 유저 x 영상 아이템
index_user_place = pd.read_csv('../Data/whyout_data/index_user_place.csv', index_col=0)
#index_user_product = pd.read_csv('../Data/whyout_data/index_user_product.csv', index_col=0)
#index_user_video = pd.read_csv('../Data/whyout_data/index_user_video.csv', index_col=0)
#normal_user_place = pd.read_csv('../Data/whyout_data/normal_index_user_place.csv', index_col=0)
#normal_user_product = pd.read_csv('../Data/whyout_data/normal_index_user_product.csv', index_col=0)
#normal_user_video = pd.read_csv('../Data/whyout_data/normal_index_user_video.csv', index_col=0)

In [57]:
# 데이터프레임을 numpy matrix로 만듦
user_product_matrix = index_user_place.values

# 사용자의 평균 평점
user_ratings_mean = np.mean(user_product_matrix, axis=1)

# 사용자-상품에 대해 사용자 평균 평점을 뺀 것
matrix_user_mean = user_product_matrix - user_ratings_mean.reshape(-1,1)

In [58]:
pd.DataFrame(matrix_user_mean, columns = index_user_place.columns).head()

,1,2,3,4,5,6,7,8,9,10,...,4688,4689,4690,4691,4692,4693,4694,4695,4696,4697
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,-0.000213,-0.000213,-0.000213,-0.000213,-0.000213,-0.000213,-0.000213,0.999787,-0.000213,-0.000213,...,-0.000213,-0.000213,-0.000213,-0.000213,-0.000213,-0.000213,-0.000213,-0.000213,-0.000213,-0.000213
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,-0.006174,-0.006174,-0.006174,-0.006174,-0.006174,1.993826,-0.006174,-0.006174,-0.006174,-0.006174,...,-0.006174,-0.006174,-0.006174,-0.006174,-0.006174,-0.006174,-0.006174,-0.006174,-0.006174,-0.006174


In [59]:
# U 행렬, sigma 행렬, V 전치 행렬을 반환.
# U, sigma, Vt = svds(matrix, k = 20)
U, sigma, Vt = randomized_svd(user_product_matrix, n_components=100)

# 특이값 벡터를 대각 행렬 형태로 변환해줌
sigma = np.diag(sigma)

In [60]:
# U, Sigma, Vt의 행렬곱을 수행하면, 다시 원본 행렬로 복원이 된다. 
# 거기에 + 사용자 평균 rating을 적용한다. 
svd_user_predicted_ratings = np.matmul(np.matmul(U,sigma), Vt) + user_ratings_mean.reshape(-1,1)

In [61]:
df_svd_preds = pd.DataFrame(svd_user_predicted_ratings, columns = index_user_place.columns)

In [64]:
df_svd_preds

,1,2,3,4,5,6,7,8,9,10,...,4688,4689,4690,4691,4692,4693,4694,4695,4696,4697
0,2.354208e-28,-1.121214e-13,1.085154e-14,6.476881e-13,4.744874e-16,1.130014e-13,-8.542704e-15,1.957934e-13,5.236425e-14,-4.479146e-28,...,0.000000,-3.580547e-15,0.000000,-4.707034e-16,5.481192e-16,-3.375283e-15,2.218068e-18,2.218068e-18,-3.396752e-16,0.000000
1,-1.004393e-28,-9.740634e-15,-1.397805e-14,7.052979e-14,-4.886936e-14,-3.443407e-15,-7.269297e-15,-1.824211e-14,-3.564687e-15,6.485971e-29,...,0.000000,2.957474e-16,0.000000,6.332255e-17,1.257965e-16,2.440725e-16,-3.241244e-19,-3.241244e-19,6.401356e-17,0.000000
2,2.129019e-04,-2.560685e-02,-6.270520e-03,-2.373288e-02,2.537590e-02,2.156037e-03,-3.831462e-03,1.934119e-01,1.163888e-02,2.129019e-04,...,0.000213,-3.897724e-04,0.000213,2.097860e-04,1.856925e-04,-4.561290e-04,2.129752e-04,2.129752e-04,2.162322e-04,0.000213
3,-1.513897e-28,-1.223959e-13,8.509093e-15,1.773391e-14,-3.755704e-15,-1.497317e-14,9.781046e-15,1.071540e-13,-3.972544e-14,-3.739071e-29,...,0.000000,1.581764e-15,0.000000,2.069677e-16,3.174078e-17,1.666681e-15,6.759560e-19,6.759560e-19,1.799034e-17,0.000000
4,6.174154e-03,2.546143e-02,1.627803e-01,1.091886e-01,-1.133910e-01,1.226199e-01,4.862004e-02,-7.713270e-03,1.115796e-01,6.174154e-03,...,0.006174,6.853285e-03,0.006174,6.262303e-03,6.183611e-03,6.811645e-03,6.173900e-03,6.173900e-03,6.119074e-03,0.006174
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31171,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
31172,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
31173,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
31174,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000


In [62]:
def recommend_movies(df_svd_preds, user_id, df, ratings_df, num_recommendations):
    
    # 원본 평점 데이터에서 user_id에 해당하는 행을 DataFrame으로 가져온다.
    user_data = ratings_df.loc[user_id]  # Series 형태로 가져오기
    #print(user_data)

    # 사용자가 이미 평가한 상품의 인덱스를 추출
    user_history_indices = user_data[user_data > 0].index.tolist()
    print(user_history_indices)

    # SVD를 통해 예측된 사용자의 상품 평점을 기반으로 상품 데이터를 정렬
    # 현재는 index로 적용이 되어있으므로 user_id - 1을 해야함.
    user_row_number = user_id - 1
    sorted_user_predictions = df_svd_preds.iloc[user_row_number].sort_values(ascending=False)

    # 사용자가 아직 평가하지 않은 상품 목록을 추출
    recommendations = df.loc[~df.index.isin(user_data[user_history_indices])]
    print(len(recommendations))
    print(len(df))

    # 예측된 상품 평점 데이터와 원본 상품 데이터를 합침
    predictions_df = pd.DataFrame(sorted_user_predictions).reset_index()
    predictions_df['index'] = predictions_df['index'].astype(int)
    recommendations = recommendations.merge(predictions_df, left_index=True, right_on='index')

    # 컬럼 이름을 바꾸고 정렬해서 반환
    recommendations = recommendations.rename(columns={user_row_number: 'Predictions'}).sort_values('Predictions', ascending=False).iloc[:num_recommendations]

    return user_history_indices, recommendations

# 함수 사용 예시
already_rated, predictions = recommend_movies(df_svd_preds, 5, place, index_user_place, 5)

['6', '789', '1776', '1916', '2417', '2707', '2716', '2719', '2786', '2789', '2800', '2801', '2809', '2811', '3167', '3412']
4694
4697


In [63]:
predictions

,idx,지역(위치정보),공간유형,주변환경,계절,테마,클릭 수,좋아요 수,북마크 수,댓글 수,index,Predictions
0,790,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,NaN,NaN,NaN,0.000657,0.000000,0.000000,0.0,789,1.057403
1,920,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,NaN,NaN,NaN,0.000329,0.000000,0.000000,0.0,919,0.407084
2,3098,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0,0,0,1,0,0,0,0,1]","[0,0,0,1,0,0,0,0]",NaN,"[0,0,0,0,0,0,0,0,0,0,0,0,0,1,0]",0.004884,0.009009,0.002513,0.0,3097,0.389790
3,2323,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,NaN,NaN,NaN,0.000845,0.000000,0.000000,0.0,2322,0.352262
4,626,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1,0,0,0,0,0,0,0,0]","[0,1,0,0,0,0,0,0]",NaN,"[1,1,1,1,1,1,1,0,0,1,0,0,0,0,1]",0.147842,0.063063,0.091709,0.0,625,0.326908
